## Dataset Download

(This notebook has been created with the intent to run it on Google Colab)

In order to download the dataset via kaggle with cmd, you have to be logged in (otherwise you don't have permission).

For that, you have to go to your kaggle account, and generate a token for API use. Then you have to place your kaggle.json file into ~/.kaggle/ folder.

In [4]:
!kaggle competitions download -c machine-learning-with-kernel-methods-2021

  0% 0.00/104k [00:00<?, ?B/s]
100% 104k/104k [00:00<00:00, 39.6MB/s]
  0% 0.00/209k [00:00<?, ?B/s]
100% 209k/209k [00:00<00:00, 63.1MB/s]
  0% 0.00/160k [00:00<?, ?B/s]
100% 160k/160k [00:00<00:00, 48.1MB/s]
  0% 0.00/104k [00:00<?, ?B/s]
100% 104k/104k [00:00<00:00, 34.2MB/s]
  0% 0.00/80.6k [00:00<?, ?B/s]
100% 80.6k/80.6k [00:00<00:00, 71.6MB/s]
  0% 0.00/80.1k [00:00<?, ?B/s]
100% 80.1k/80.1k [00:00<00:00, 62.6MB/s]
  0% 0.00/13.7k [00:00<?, ?B/s]
100% 13.7k/13.7k [00:00<00:00, 27.4MB/s]
  0% 0.00/209k [00:00<?, ?B/s]
100% 209k/209k [00:00<00:00, 27.9MB/s]
  0% 0.00/80.2k [00:00<?, ?B/s]
100% 80.2k/80.2k [00:00<00:00, 76.7MB/s]
  0% 0.00/160k [00:00<?, ?B/s]
100% 160k/160k [00:00<00:00, 52.0MB/s]
  0% 0.00/103k [00:00<?, ?B/s]
100% 103k/103k [00:00<00:00, 84.1MB/s]
  0% 0.00/12.6k [00:00<?, ?B/s]
100% 12.6k/12.6k [00:00<00:00, 10.4MB/s]
  0% 0.00/161k [00:00<?, ?B/s]
100% 161k/161k [00:00<00:00, 48.9MB/s]
  0% 0.00/208k [00:00<?, ?B/s]
100% 208k/208k [00:00<00:00, 61.4MB/s]
  0% 

In [33]:
!unzip Xtr0_mat100.csv.zip

Archive:  Xtr0_mat100.csv.zip
  inflating: Xtr0_mat100.csv         


In [7]:
import cvxpy as cp
import numpy as np
import pandas as pd

This is a simple test using only Xtr0_mat100.csv (there is 3 of them in total)

I split the 2000 entry between 1800 for train and 200 for test.

In [83]:
data = pd.read_csv('Xtr0_mat100.csv', header=None)
test_array = np.array([data[0][k].split() for k in range(len(data[0]))]).astype(float).reshape(-1,100)
train_data = test_array[:1800,:]
test_data = test_array[1800:,:]
labels_data = pd.read_csv('Ytr0.csv', header=None)
labels = labels_data[1][1:].values.astype(float) * 2 - 1
train_labels = labels[:1800]
test_labels = labels[1800:]

I use a simple Linear Logic Regression to predict data

In [85]:
OLS = np.linalg.inv(train_data.T @ train_data + 0.1 * np.eye(train_data.shape[1])) @ train_data.T @ train_labels

In [113]:
predict = ((test_data @ OLS) >= 0) * 2 - 1
predict

array([-1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1,  1,  1,  1,  1,  1, -1,
       -1,  1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1,
       -1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1, -1, -1, -1,
        1, -1,  1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1,
        1, -1,  1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1,  1, -1,
       -1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1,
        1, -1,  1, -1,  1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1, -1, -1,
       -1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1, -1, -1,  1,
       -1,  1, -1, -1, -1,  1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        1,  1,  1,  1,  1,  1, -1,  1, -1,  1, -1, -1,  1,  1,  1, -1,  1,
        1,  1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1])

In [130]:
accuracy = (predict != test_labels).sum() / len(test_labels)
print("ACCURACY IS", accuracy)

print("Keep in mind that random prediction gives", ((np.random.randint(2, size=200) * 2 - 1) != test_labels).sum() / 200, "...")

ACCURACY IS  0.45
Keep in mind that random prediction gives 0.48 ...


As you can see results are pretty bad.

Then let's try Kernel Ridge Regression with Gaussian Kernel

We build the Gram Matrix using scipy.

In [132]:
from scipy.spatial import distance

In [167]:
K_sigma = 1
lamb_regularization = 10000
K_gram = np.exp(-distance.cdist(train_data, train_data) ** 2 / (2 * K_sigma ** 2))
K_gram.shape

(1800, 1800)

In [168]:
alpha = np.linalg.pinv(K_gram + lamb_regularization * np.eye(K_gram.shape[0])) @ train_labels

In [169]:
K_test = np.exp(-distance.cdist(test_data, train_data) ** 2 / (2 * K_sigma ** 2))
predict = ((alpha * K_test).sum(axis=1) >= 0) * 2 - 1
predict

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [170]:
accuracy_gauss = (predict != test_labels).sum() / len(test_labels)
print("ACCURACY IS", accuracy_gauss)

ACCURACY IS 0.505


Results are better but still terrible.

To do: SVM with Gaussian Kernel

Please let me know if you see blatant errors in my code.